# Customer Insights Dashboard

In [1]:
# Understand customer behavior and preferences to enhance customer experience.
# Customer segmentation by age and gender (Stacked bar chart)
# Purchase frequency over time (Line chart)
# Average purchase value by customer segment (Bar chart)
# Customer demographic and purchase data, such as age, gender, location, and purchase history.

In [2]:
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA  

In [3]:
# Define the path to the zip file
zip_file_path = 'Analyzing Customer Spending Habits.zip'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract the CSV file from the zip
    zip_ref.extractall()  # This will extract all files into the current directory

# Load the CSV file into a pandas DataFrame
csv_file_path = 'SalesForCourse_quizz_table.csv'
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(df.head())
print(df.describe())
print(df.info())

   index      Date    Year     Month  Customer Age Customer Gender  \
0      0  02/19/16  2016.0  February          29.0               F   
1      1  02/20/16  2016.0  February          29.0               F   
2      2  02/27/16  2016.0  February          29.0               F   
3      3  03/12/16  2016.0     March          29.0               F   
4      4  03/12/16  2016.0     March          29.0               F   

         Country       State Product Category     Sub Category  Quantity  \
0  United States  Washington      Accessories  Tires and Tubes       1.0   
1  United States  Washington         Clothing           Gloves       2.0   
2  United States  Washington      Accessories  Tires and Tubes       3.0   
3  United States  Washington      Accessories  Tires and Tubes       2.0   
4  United States  Washington      Accessories  Tires and Tubes       3.0   

   Unit Cost  Unit Price   Cost  Revenue  Column1  
0      80.00  109.000000   80.0    109.0      NaN  
1      24.50   28.

In [4]:
# Handle missing values

In [5]:
# Step 1: Handling Missing Values
# Drop rows where critical columns are missing (e.g., 'Date', 'Customer Age', 'Revenue')
df.dropna(subset=['Date', 'Customer Age', 'Revenue'], inplace=True)

# Fill missing values for other columns (like 'Column1')
df['Column1'] = df['Column1'].fillna(0)  # Updated to avoid inplace method

# Step 2: Data Type Conversion
# Convert 'Date' to a datetime format for better handling of time series data
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')

# Step 3: Create New Features
# Create a 'Customer Segment' based on age groups (e.g., <25, 25-40, 40-60, 60+)
df['Customer Segment'] = pd.cut(df['Customer Age'], 
                                bins=[0, 25, 40, 60, 100], 
                                labels=['<25', '25-40', '40-60', '60+'])

# Step 4: Calculate Total Purchase Value (Revenue)
# Assuming the 'Revenue' column already represents total purchase value per transaction
df['Total Purchase Value'] = df['Quantity'] * df['Unit Price']

# Step 5: Grouping Data by Demographics and Purchase History
# Aggregate the data by Customer Segment, Gender, and Date
customer_data = df.groupby(
    ['Customer Segment', 'Customer Gender', 'Date'], observed=True  # Add observed=True to avoid warning
).agg({
    'Total Purchase Value': 'sum',    # Total revenue per segment
    'Quantity': 'sum',                # Total items purchased
    'Cost': 'sum',                    # Total cost
    'Revenue': 'sum',                 # Total revenue
    'Customer Age': 'mean'            # Average age in the segment
}).reset_index()

# Step 6: Save the Processed Data to a CSV File
# This file can now be used for Tableau
output_csv_file = 'processed_customer_spending_data.csv'
customer_data.to_csv(output_csv_file, index=False)

print(f"Processed dataset saved to '{output_csv_file}'")

Processed dataset saved to 'processed_customer_spending_data.csv'
